# Here is the refactored solution. I have separated the concerns into three distinct layers:
1.  **The Data Contract:** explicit `dataclasses` defining exactly what goes in and comes out.
2.  **The Engine:** A purely mathematical class (`AlphaEngine`) containing the logic, with no widget/plotting dependencies.
3.  **The UI:** A cleaned-up dashboard function that simply sends inputs to the Engine and visualizes the Output

### Following is the reverse chronological fix log (most recent entry is at the top )

v38  
Added Momentum and Pullback strategies

v37  
For an automated bot (and for rigorous scientific testing), **Silent Auto-Correction (Clamping)** is dangerous. If the bot asks for "2080" and receives data for "2025", it creates "Data Hallucinations."

We need to replace the "Clamping" logic with **"Strict Validation" logic**.


v36  
The step `self.df_close = df_ohlcv['Adj Close'].unstack(level=0)` is an expensive "Pivot" operation. It takes a "Tall and Skinny" table (1 million rows) and reshapes it into a "Short and Wide" matrix (Dates x Tickers). Pandas hates doing this repeatedly.

Yes, we can pre-compute this matrix and pass it in, just like we did with the featur

v35  
This is a great optimization step. In data science, this is called **"Memoization"** or **"Caching"**—do the heavy math once, save it, and reuse it.

To achieve this, we need to modify the **`AlphaEngine` constructor** to accept pre-calculated features, and update the **`plot_walk_forward_analyzer`** to pass them down.


v34  
I have refactored **SECTION D (`AlphaEngine.run`)** to implement the "Decision Anchor" logic, and **SECTION E (`plot_walk_forward_analyzer`)** to implement the new "Timeline" UI layout and renaming.

### Key Changes:
1.  **Engine Logic:** The `start_date` input is now treated as the **Decision Date (T0)**. The engine calculates backward for the Lookback period and forward for the Holding period.
2.  **Universe Selection:** Tickers are now filtered based on liquidity **on the Decision Date**, not the start of history.
3.  **UI Layout:** Inputs are arranged horizontally: `[Lookback] <-> [Decision Date] <-> [Holding]`.
4.  **Renaming:** "Metric" is now **"Strategy"**.


v33  
**Action Item to make it perfect:**
Change the "Fwd Gain" calculation in `AlphaEngine.run` to skip one day, or accept that your results are slightly optimistic due to the "Overnight Gap" bias.

```
To see the full dataframe of all tickers (both those that passed and those that failed) for a specific date, we need to capture a snapshot of the universe inside the `_get_eligible_universe` method.

I have updated the **`AlphaEngine`** class below.
```

```
To verify that the relative percentile logic is working, we can modify the `AlphaEngine` to report exactly **how the cutoff was calculated** for the specific start date.

We want to see evidence that:
1.  In earlier years (e.g., 2005), the volume cutoff is lower (e.g., $200k).
2.  In later years (e.g., 2024), the volume cutoff is higher (e.g., $5M).

Here is the updated `AlphaEngine` and `UI` code. I have added a **"Audit Log"** feature. When you run the tool, it will now print exactly what the Dollar Volume Threshold was for that specific day.
```

```
The best way to solve this is to switch from a **Fixed Dollar Threshold** (e.g., "$1 Million") to a **Relative Percentile Threshold** (e.g., "Top 50% of the market").

In 2004, a stock trading $200k might have been in the top 50% of liquid stocks. In 2024, that same $200k is illiquid garbage. Using a percentile automatically adjusts for inflation and market growth over time.

Here is how to modify your code to support this.
```

```
To fix this, we need to pass the **actual** calculated start date (the trading day the engine "snapped" to) back from the `AlphaEngine` to the UI. Then, the UI can compare the *Requested Date* vs. the *Actual Date* and display the warning message if they differ.

Here is the plan:
1.  **Update `EngineOutput`**: Add a `start_date` field to the dataclass.
2.  **Update `AlphaEngine.run`**: Populate this new field with `safe_start_date`.
3.  **Update `plot_walk_forward_analyzer`**: Add logic to compare the user's input date with the engine's returned date and print the "Info" message if they are different.

Here is the updated code (Sections C, D, and E have changed):
```

```
I have updated the `AlphaEngine.run` method. specifically inside the `if inputs.mode == 'Manual List':` block. It now iterates through every manual ticker and performs two checks:
1.  **Existence**: Is the ticker in the database?
2.  **Availability**: Does the ticker have a valid price on the specific `Start Date`?

If any ticker fails, it compiles a specific error message explaining why (e.g., "No price data on start date") and aborts the calculation immediately.  
```

```
The `snapshot_df` contains **every single feature** calculated by your `generate_features` function for that specific day, plus the new audit columns we added.

Here is exactly what is inside that DataFrame:

### 1. The Core Features (from `generate_features`)
*   **`TR`**: True Range
*   **`ATR`**: Average True Range
*   **`ATRP`**: Average True Range Percent (Volatility)
*   **`RollingStalePct`**: How often the price didn't move or volume was 0.
*   **`RollMedDollarVol`**: Median Daily Dollar Volume (Liquidity).
*   **`RollingSameVolCount`**: Data quality check for repeated volume numbers.

### 2. The Audit Columns (Added during filtering)
*   **`Calculated_Cutoff`**: The specific dollar amount required to pass on that day.
*   **`Passed_Vol_Check`**: `True` if the ticker met the liquidity requirement.
*   **`Passed_Final`**: `True` if it passed **all** checks (Liquidity + Stale + Quality).

=========================================

Here are the formulas translated directly into the Python `pandas` code used in your `generate_features` function.

I have simplified the code slightly to assume a single ticker context (removing the `groupby` wrapper) so you can see the raw math clearly.

### 1. True Range (TR)
Calculates the maximum of the three price differences.

prev_close = df_ohlcv['Adj Close'].shift(1)

# The three components
diff1 = df_ohlcv['Adj High'] - df_ohlcv['Adj Low']
diff2 = (df_ohlcv['Adj High'] - prev_close).abs()
diff3 = (df_ohlcv['Adj Low'] - prev_close).abs()

# Taking the max of the three
tr = pd.concat([diff1, diff2, diff3], axis=1).max(axis=1)

### 2. Average True Range (ATR)
Uses an Exponential Weighted Mean (EWM) with a specific alpha smoothing factor.

# N = atr_period (e.g., 14)
# alpha = 1 / N
atr = tr.ewm(alpha=1/14, adjust=False).mean()

### 3. ATR Percent (ATRP)
Simple division to normalize volatility.

atrp = atr / df_ohlcv['Adj Close']

### 4. Rolling Stale Percentage
Checks if volume is 0 OR if High equals Low (price didn't move), then averages that 1 or 0 signal over the window.

# 1. Define the Stale Signal (1 for stale, 0 for active)
is_stale = np.where(
    (df_ohlcv['Volume'] == 0) | (df_ohlcv['Adj High'] == df_ohlcv['Adj Low']), 
    1,  
    0
)

# 2. Calculate average over window (W=252)
rolling_stale_pct = pd.Series(is_stale).rolling(window=252).mean()

### 5. Rolling Median Dollar Volume
Calculates raw dollar volume, then finds the median over the window.

# 1. Calculate Daily Dollar Volume
dollar_volume = df_ohlcv['Adj Close'] * df_ohlcv['Volume']

# 2. Get Median over window (W=252)
roll_med_dollar_vol = dollar_volume.rolling(window=252).median()

### 6. Rolling Same Volume Count
Checks if today's volume is exactly the same as yesterday's (a sign of bad data), then sums those occurrences.

# 1. Check if Volume(t) - Volume(t-1) equals 0
# .diff() calculates current row minus previous row
has_same_volume = (df_ohlcv['Volume'].diff() == 0).astype(int)

# 2. Sum the errors over window (W=252)
rolling_same_vol_count = has_same_volume.rolling(window=252).sum()

```

================================  

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import ipywidgets as widgets
import pprint

from dataclasses import dataclass, field
from typing import List, Dict, Optional, Any, Union
from collections import Counter
from datetime import datetime, date
from pandas.testing import assert_series_equal

# pd.set_option('display.max_rows', None)  display all rows
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 50)
pd.set_option('display.precision', 4)

# ==============================================================================
# SECTION A: CORE HELPER FUNCTIONS & FEATURE GENERATION
# (Unchanged from previous version)
# ==============================================================================
# ... (Keep generate_features, calculate_gain, calculate_sharpe, 
#      calculate_sharpe_atr, calculate_buy_and_hold_performance as is) ...

def generate_features(df_ohlcv: pd.DataFrame, atr_period: int = 14, quality_window: int = 252, quality_min_periods: int = 126) -> pd.DataFrame:
    # 1. Sort and Group
    if not df_ohlcv.index.is_monotonic_increasing: 
        df_ohlcv = df_ohlcv.sort_index()
    grouped = df_ohlcv.groupby(level='Ticker')
    
    # 2. ATR Calculation (Existing)
    prev_close = grouped['Adj Close'].shift(1)
    tr = pd.concat([
        df_ohlcv['Adj High'] - df_ohlcv['Adj Low'], 
        abs(df_ohlcv['Adj High'] - prev_close), 
        abs(df_ohlcv['Adj Low'] - prev_close)
    ], axis=1).max(axis=1, skipna=False)
    
    atr = tr.groupby(level='Ticker').transform(lambda x: x.ewm(alpha=1/atr_period, adjust=False).mean())
    atrp = (atr / df_ohlcv['Adj Close']).replace([np.inf, -np.inf], np.nan)
    
    # 3. --- NEW: MOMENTUM / RETURN FEATURES ---
    # We calculate percentage change for specific windows useful for Pullbacks (3D, 5D) and Trends (21D)
    # Note: We use grouped.pct_change to respect Ticker boundaries
    roc_1 = grouped['Adj Close'].pct_change(1)
    roc_3 = grouped['Adj Close'].pct_change(3)
    roc_5 = grouped['Adj Close'].pct_change(5)
    roc_10 = grouped['Adj Close'].pct_change(10)
    roc_21 = grouped['Adj Close'].pct_change(21)

    indicator_df = pd.DataFrame({
        'TR': tr, 
        'ATR': atr, 
        'ATRP': atrp,
        'ROC_1': roc_1,
        'ROC_3': roc_3,
        'ROC_5': roc_5,
        'ROC_10': roc_10,
        'ROC_21': roc_21
    })

    # 4. Quality/Liquidity Features (Existing)
    quality_temp_df = pd.DataFrame({
        'IsStale': np.where((df_ohlcv['Volume'] == 0) | (df_ohlcv['Adj High'] == df_ohlcv['Adj Low']), 1, 0), 
        'DollarVolume': df_ohlcv['Adj Close'] * df_ohlcv['Volume'], 
        'HasSameVolume': (grouped['Volume'].diff() == 0).astype(int)
    }, index=df_ohlcv.index)
    
    rolling_result = quality_temp_df.groupby(level='Ticker').rolling(window=quality_window, min_periods=quality_min_periods).agg({
        'IsStale': 'mean', 
        'DollarVolume': 'median', 
        'HasSameVolume': 'sum'
    }).rename(columns={
        'IsStale': 'RollingStalePct', 
        'DollarVolume': 'RollMedDollarVol', 
        'HasSameVolume': 'RollingSameVolCount'
    }).reset_index(level=0, drop=True)
    
    # 5. Merge
    return pd.concat([indicator_df, rolling_result], axis=1)

def calculate_gain(price_series): 
    if price_series.dropna().shape[0] < 2: return np.nan
    return (price_series.ffill().iloc[-1] / price_series.bfill().iloc[0]) - 1

def calculate_sharpe(return_series):
    if return_series.dropna().shape[0] < 2: return np.nan
    std = return_series.std()
    return (return_series.mean() / std * np.sqrt(252)) if std > 0 else 0.0

def calculate_sharpe_atr(return_series, atrp_series):
    if return_series.dropna().shape[0] < 2 or atrp_series.dropna().empty: return np.nan
    mean_atrp = atrp_series.mean()
    return (return_series.mean() / mean_atrp) if mean_atrp > 0 else 0.0

def calculate_buy_and_hold_performance(df_close, features_df, tickers, start_date, end_date):
    if not tickers: return pd.Series(dtype=float), pd.Series(dtype=float), pd.Series(dtype=float)
    ticker_counts = Counter(tickers)
    initial_weights = pd.Series({t: c / len(tickers) for t, c in ticker_counts.items()})
    prices_raw = df_close[initial_weights.index.tolist()].loc[start_date:end_date]
    if prices_raw.dropna(how='all').empty: return pd.Series(dtype=float), pd.Series(dtype=float), pd.Series(dtype=float)
    prices_norm = prices_raw.div(prices_raw.bfill().iloc[0])
    weighted_growth = prices_norm.mul(initial_weights, axis='columns')
    value_series = weighted_growth.sum(axis=1)
    return_series = value_series.ffill().pct_change()
    full_idx = pd.MultiIndex.from_product([initial_weights.index.tolist(), return_series.index], names=['Ticker', 'Date'])
    feat_subset = features_df.reindex(full_idx)['ATRP'].unstack(level='Ticker')
    atrp_series = (weighted_growth.div(value_series, axis='index').align(feat_subset, join='inner', axis=1)[0] * weighted_growth.div(value_series, axis='index').align(feat_subset, join='inner', axis=1)[1]).sum(axis=1)
    return value_series, return_series, atrp_series

# ==============================================================================
# SECTION B: METRIC REGISTRY (UPDATED)
# ==============================================================================

# Existing Metrics
def metric_price(d): return calculate_gain(d['calc_close'])
def metric_sharpe(d): 
    r = d['daily_returns']
    return (r.mean() / r.std() * np.sqrt(252)).replace([np.inf, -np.inf], np.nan).fillna(0)
def metric_sharpe_atr(d):
    return (d['daily_returns'].mean() / d['atrp']).replace([np.inf, -np.inf], np.nan).fillna(0)

METRIC_REGISTRY = {
    'Price': metric_price,
    'Sharpe': metric_sharpe,
    'Sharpe (ATR)': metric_sharpe_atr,
}

# --- NEW: MOMENTUM (High % Return = Rank 1) ---
# "Buy Strength"
METRIC_REGISTRY['Momentum 1D'] = lambda d: d['roc_1']
METRIC_REGISTRY['Momentum 3D'] = lambda d: d['roc_3']
METRIC_REGISTRY['Momentum 5D'] = lambda d: d['roc_5']
METRIC_REGISTRY['Momentum 10D'] = lambda d: d['roc_10']
METRIC_REGISTRY['Momentum 1M'] = lambda d: d['roc_21']

# --- NEW: PULLBACK (Low/Negative % Return = Rank 1) ---
# We invert the value (-) so the deepest drop becomes the largest positive number
# Example: -10% becomes +0.10, which ranks higher than -2% (+0.02)
METRIC_REGISTRY['Pullback 1D'] = lambda d: -d['roc_1']
METRIC_REGISTRY['Pullback 3D'] = lambda d: -d['roc_3']
METRIC_REGISTRY['Pullback 5D'] = lambda d: -d['roc_5']
METRIC_REGISTRY['Pullback 10D'] = lambda d: -d['roc_10']
METRIC_REGISTRY['Pullback 1M'] = lambda d: -d['roc_21']

# ==============================================================================
# SECTION C: DATA CONTRACTS (The API)
# Updated EngineOutput to include actual start_date
# ==============================================================================

@dataclass
class EngineInput:
    mode: str
    start_date: pd.Timestamp
    calc_period: int
    fwd_period: int
    metric: str
    benchmark_ticker: str
    rank_start: int = 1
    rank_end: int = 10
    quality_thresholds: Dict[str, float] = field(default_factory=lambda: {'min_median_dollar_volume': 1_000_000, 'max_stale_pct': 0.05, 'max_same_vol_count': 10})
    manual_tickers: List[str] = field(default_factory=list)
    debug: bool = False

@dataclass
class EngineOutput:
    portfolio_series: pd.Series
    benchmark_series: pd.Series
    normalized_plot_data: pd.DataFrame
    tickers: List[str]
    initial_weights: pd.Series
    perf_metrics: Dict[str, float]
    results_df: pd.DataFrame
    start_date: pd.Timestamp # <--- NEW FIELD: The actual trading start date used
    calc_end_date: pd.Timestamp
    viz_end_date: pd.Timestamp
    error_msg: Optional[str] = None
    debug_data: Optional[Dict[str, Any]] = None

# ==============================================================================
# SECTION D: THE ALPHA ENGINE (The "Brain")
# Updated for Decision-Anchor Logic
# ==============================================================================

class AlphaEngine:
    # 1. Add 'df_close_wide' as an optional argument
    def __init__(self, df_ohlcv: pd.DataFrame, features_df: pd.DataFrame = None, df_close_wide: pd.DataFrame = None, master_ticker: str = 'SPY'):
        print("--- ⚙️ Initializing AlphaEngine ---")
        
        # Handle Features
        if features_df is not None:
            # print("⚡ Using Pre-Computed Features") # Optional: Reduce clutter
            self.features_df = features_df
        else:
            print("🐢 Calculating Features from scratch...")
            self.features_df = generate_features(df_ohlcv)
            
        # 2. Handle Price Matrix (The Optimization)
        if df_close_wide is not None:
            print("⚡ Using Pre-Computed Price Matrix (Instant Load)")
            self.df_close = df_close_wide
        else:
            print("🐢 Pivoting Price Data (Slow)...")
            self.df_close = df_ohlcv['Adj Close'].unstack(level=0)
        
        # Master Ticker Check
        if master_ticker not in self.df_close.columns:
            master_ticker = self.df_close.columns[0]
            print(f"Warning: Master ticker not found. Using {master_ticker}")
            
        self.trading_calendar = self.df_close[master_ticker].dropna().index.unique().sort_values()
        # print("✅ AlphaEngine Ready.")

    def run(self, inputs: EngineInput) -> EngineOutput:
        # --- A. Validate Dates (STRICT VALIDATION MODE) ---
        
        # 1. Establish Boundaries
        # The earliest possible decision requires 'Lookback' days of history before it.
        # The latest possible decision is the last day of data.
        
        if len(self.trading_calendar) <= inputs.calc_period:
             return self._error_result(f"Dataset too small. Has {len(self.trading_calendar)} days, need at least {inputs.calc_period + 1}.")

        min_decision_date = self.trading_calendar[inputs.calc_period]
        max_decision_date = self.trading_calendar[-1]

        # 2. Check: Is the requested date too early?
        # We allow "snapping" to valid trading days, but only if within the valid range.
        if inputs.start_date < self.trading_calendar[0]:
             return self._error_result(f"❌ Date Out of Range (Too Early). Earliest valid decision date (with {inputs.calc_period} day lookback) is {min_decision_date.date()}.")
        
        if inputs.start_date < min_decision_date:
             return self._error_result(f"❌ Not enough history. Decision date {inputs.start_date.date()} does not have {inputs.calc_period} previous days. Try {min_decision_date.date()} or later.")

        # 3. Check: Is the requested date too late?
        if inputs.start_date > max_decision_date:
             return self._error_result(f"❌ Date Out of Range (Too Late). Dataset ends on {max_decision_date.date()}. Please choose a date on or before that.")

        # 4. If we pass checks, Find the Index
        # searchsorted finds the insertion point (snaps forward to nearest trading day if input is a weekend)
        decision_idx = self.trading_calendar.searchsorted(inputs.start_date)
        
        # Double check to ensure searchsorted didn't push us off the edge (edge case for date > last_date but close)
        if decision_idx >= len(self.trading_calendar):
             decision_idx = len(self.trading_calendar) - 1

        # 5. Set Indices
        start_idx = decision_idx - inputs.calc_period
        
        # Calculate Forward (Holding End)
        end_idx = decision_idx + inputs.fwd_period
        if end_idx >= len(self.trading_calendar):
            end_idx = len(self.trading_calendar) - 1

        # 6. Set Safe Dates
        safe_start_date = self.trading_calendar[start_idx]      
        safe_calc_end_date = self.trading_calendar[decision_idx] 
        safe_viz_end_date = self.trading_calendar[end_idx] 

        # --- B. Select Tickers ---
        tickers_to_trade = []
        results_table = pd.DataFrame()
        debug_dict = {}
        audit_info = {} 

        if inputs.mode == 'Manual List':
            validation_errors = []
            valid_tickers = []
            for t in inputs.manual_tickers:
                if t not in self.df_close.columns:
                    validation_errors.append(f"❌ {t}: Ticker not found.")
                    continue
                if pd.isna(self.df_close.at[safe_start_date, t]):
                    validation_errors.append(f"⚠️ {t}: No price data on start date.")
                    continue
                valid_tickers.append(t)
            
            if validation_errors: return self._error_result("\n".join(validation_errors))
            if not valid_tickers: return self._error_result("No valid tickers.")
            tickers_to_trade = valid_tickers
            results_table = pd.DataFrame(index=valid_tickers)
            
        else: # Ranking Mode
            # CRITICAL UPDATE: Filter Universe based on the DECISION DATE (safe_calc_end_date)
            # We want stocks that are liquid *now*, not 6 months ago.
            eligible_tickers = self._get_eligible_universe(safe_calc_end_date, inputs.quality_thresholds, audit_info)
            debug_dict['audit_liquidity'] = audit_info 
            
            if not eligible_tickers: return self._error_result("No tickers passed quality filters.")
            
            # --- UPDATED INGREDIENTS EXTRACTION ---
            # 1. Get Price Data
            calc_close = self.df_close.loc[safe_start_date:safe_calc_end_date, eligible_tickers]
            
            # 2. Get Feature Data (Specifically for the DECISION DATE)
            # We use .xs() to grab the row corresponding to safe_calc_end_date for all tickers
            feat_slice_current = self.features_df.xs(safe_calc_end_date, level='Date').reindex(eligible_tickers)
            
            # 3. Get Lookback Means (for ATRP)
            idx_product = pd.MultiIndex.from_product([eligible_tickers, calc_close.index], names=['Ticker', 'Date'])
            feat_slice_period = self.features_df.reindex(idx_product).dropna(how='all')
            atrp_mean = feat_slice_period.groupby(level='Ticker')['ATRP'].mean()

            # Extract the specific feature slice for the decision date
            feat_slice_current = self.features_df.xs(safe_calc_end_date, level='Date').reindex(eligible_tickers)

            # 4. Populate Ingredients
            ingredients = { 
                'calc_close': calc_close, 
                'daily_returns': calc_close.ffill().pct_change(), 
                'atrp': atrp_mean,
                # --- NEW WIRES CONNECTED HERE ---
                'roc_1': feat_slice_current['ROC_1'],
                'roc_3': feat_slice_current['ROC_3'],
                'roc_5': feat_slice_current['ROC_5'],
                'roc_10': feat_slice_current['ROC_10'],
                'roc_21': feat_slice_current['ROC_21']
            }

            if inputs.metric not in METRIC_REGISTRY: return self._error_result(f"Strategy '{inputs.metric}' not found.")
            metric_vals = METRIC_REGISTRY[inputs.metric](ingredients)
            sorted_tickers = metric_vals.sort_values(ascending=False)
            
            start_r = max(0, inputs.rank_start - 1)
            end_r = inputs.rank_end
            tickers_to_trade = sorted_tickers.iloc[start_r:end_r].index.tolist()
            if not tickers_to_trade: return self._error_result("No tickers generated from ranking.")

            results_table = pd.DataFrame({
                'Rank': range(inputs.rank_start, inputs.rank_start + len(tickers_to_trade)),
                'Ticker': tickers_to_trade,
                'Strategy Value': sorted_tickers.loc[tickers_to_trade].values
            }).set_index('Ticker')

        # --- C. Performance Calculations ---
        # Run simulation for the full timeline (Lookback + Holding)
        p_val, p_ret, p_atrp = calculate_buy_and_hold_performance(self.df_close, self.features_df, tickers_to_trade, safe_start_date, safe_viz_end_date)
        b_val, b_ret, b_atrp = calculate_buy_and_hold_performance(self.df_close, self.features_df, [inputs.benchmark_ticker], safe_start_date, safe_viz_end_date)

        # --- D. Final Metrics (With 1-Day Lag Fix) ---
        plot_data = self.df_close[list(set(tickers_to_trade))].loc[safe_start_date:safe_viz_end_date]
        if not plot_data.empty: plot_data = plot_data / plot_data.bfill().iloc[0]
        
        calc_end_ts = safe_calc_end_date
        metrics = {}
        get_gain = lambda s: (s.iloc[-1] / s.iloc[0]) - 1 if len(s) > 0 else 0

        # Helper for Lagged Forward Slice (T+1)
        def get_lagged_fwd(series, boundary_date):
            slice_raw = series.loc[boundary_date:]
            if len(slice_raw) > 1:
                return slice_raw.iloc[1:] # Skip the boundary day
            return pd.Series(dtype=float)

        # 1. Portfolio Metrics
        metrics['full_p_gain'] = get_gain(p_val)
        metrics['calc_p_gain'] = get_gain(p_val.loc[:calc_end_ts])
        
        p_val_fwd_lagged = get_lagged_fwd(p_val, calc_end_ts)
        metrics['fwd_p_gain'] = get_gain(p_val_fwd_lagged)

        # Sharpe (ATR)
        metrics['full_p_sharpe_atr'] = calculate_sharpe_atr(p_ret, p_atrp)
        metrics['calc_p_sharpe_atr'] = calculate_sharpe_atr(p_ret.loc[:calc_end_ts], p_atrp.loc[p_ret.loc[:calc_end_ts].index])
        
        # Fwd Sharpe Prep
        p_ret_fwd_lagged = p_ret.loc[calc_end_ts:].iloc[2:] if len(p_ret.loc[calc_end_ts:]) > 2 else pd.Series(dtype=float)
        p_atrp_fwd_lagged = p_atrp.reindex(p_ret_fwd_lagged.index)
        metrics['fwd_p_sharpe_atr'] = calculate_sharpe_atr(p_ret_fwd_lagged, p_atrp_fwd_lagged)

        # Standard Sharpe
        metrics['full_p_sharpe'] = calculate_sharpe(p_ret)
        metrics['calc_p_sharpe'] = calculate_sharpe(p_ret.loc[:calc_end_ts])
        metrics['fwd_p_sharpe'] = calculate_sharpe(p_ret_fwd_lagged)
        
        # 2. Benchmark Metrics
        if not b_ret.empty:
            metrics['full_b_gain'] = get_gain(b_val)
            metrics['calc_b_gain'] = get_gain(b_val.loc[:calc_end_ts])
            
            b_val_fwd_lagged = get_lagged_fwd(b_val, calc_end_ts)
            metrics['fwd_b_gain'] = get_gain(b_val_fwd_lagged)
            
            metrics['full_b_sharpe_atr'] = calculate_sharpe_atr(b_ret, b_atrp)
            metrics['calc_b_sharpe_atr'] = calculate_sharpe_atr(b_ret.loc[:calc_end_ts], b_atrp.loc[b_ret.loc[:calc_end_ts].index])
            
            b_ret_fwd_lagged = b_ret.loc[calc_end_ts:].iloc[2:] if len(b_ret.loc[calc_end_ts:]) > 2 else pd.Series(dtype=float)
            b_atrp_fwd_lagged = b_atrp.reindex(b_ret_fwd_lagged.index)
            metrics['fwd_b_sharpe_atr'] = calculate_sharpe_atr(b_ret_fwd_lagged, b_atrp_fwd_lagged)

            metrics['full_b_sharpe'] = calculate_sharpe(b_ret)
            metrics['calc_b_sharpe'] = calculate_sharpe(b_ret.loc[:calc_end_ts])
            metrics['fwd_b_sharpe'] = calculate_sharpe(b_ret_fwd_lagged)

        if not plot_data.empty: results_table['Fwd Gain'] = (plot_data.iloc[-1] / plot_data.loc[calc_end_ts]) - 1
        ticker_counts = Counter(tickers_to_trade)
        weights = pd.Series({t: c/len(tickers_to_trade) for t, c in ticker_counts.items()})

        if inputs.debug:
            trace_df = plot_data.copy()
            trace_df.columns = [f'Norm_Price_{c}' for c in trace_df.columns]
            trace_df['Norm_Price_Portfolio'] = p_val
            if not b_val.empty: trace_df[f'Norm_Price_Benchmark_{inputs.benchmark_ticker}'] = b_val
            debug_dict['portfolio_trace'] = trace_df

        return EngineOutput(
            portfolio_series=p_val, benchmark_series=b_val, normalized_plot_data=plot_data,
            tickers=tickers_to_trade, initial_weights=weights, perf_metrics=metrics,
            results_df=results_table, start_date=safe_start_date,
            calc_end_date=safe_calc_end_date, viz_end_date=safe_viz_end_date, 
            error_msg=None, debug_data=debug_dict
        )

    def _get_eligible_universe(self, date_ts, thresholds, audit_container=None):
        avail_dates = self.features_df.index.get_level_values('Date').unique().sort_values()
        valid_dates = avail_dates[avail_dates <= date_ts]
        if valid_dates.empty: return []
        day_features = self.features_df.xs(valid_dates[-1], level='Date')

        vol_cutoff = thresholds.get('min_median_dollar_volume', 0)
        percentile_used = "N/A"
        dynamic_val = 0
        
        if 'min_liquidity_percentile' in thresholds:
            percentile_used = thresholds['min_liquidity_percentile']
            dynamic_val = day_features['RollMedDollarVol'].quantile(percentile_used)
            vol_cutoff = max(vol_cutoff, dynamic_val)

        mask = (
            (day_features['RollMedDollarVol'] >= vol_cutoff) &
            (day_features['RollingStalePct'] <= thresholds['max_stale_pct']) &
            (day_features['RollingSameVolCount'] <= thresholds['max_same_vol_count'])
        )

        if audit_container is not None:
            audit_container['date'] = valid_dates[-1]
            audit_container['total_tickers_available'] = len(day_features)
            audit_container['percentile_setting'] = percentile_used
            audit_container['percentile_value_usd'] = dynamic_val
            audit_container['final_cutoff_usd'] = vol_cutoff
            audit_container['tickers_passed'] = mask.sum()
            snapshot = day_features.copy()
            snapshot['Calculated_Cutoff'] = vol_cutoff
            snapshot['Passed_Vol_Check'] = snapshot['RollMedDollarVol'] >= vol_cutoff
            snapshot['Passed_Final'] = mask
            snapshot = snapshot.sort_values('RollMedDollarVol', ascending=False)
            audit_container['universe_snapshot'] = snapshot

        return day_features[mask].index.tolist()

    def _error_result(self, msg):
        return EngineOutput(pd.Series(dtype=float), pd.Series(dtype=float), pd.DataFrame(), [], pd.Series(dtype=float), {}, pd.DataFrame(), pd.Timestamp.min, pd.Timestamp.min, pd.Timestamp.min, msg)

# ==============================================================================
# SECTION E: THE UI (Visualization)
# Added precomputed_features
# Updated for Timeline Layout
# Update this function to read the audit data from the `debug_data` and print it nicely.
# Updated print logic to detect date shift
# Fixed EngineInput argument mapping
# ==============================================================================

def plot_walk_forward_analyzer(df_ohlcv, 
                               precomputed_features=None,
                               precomputed_close=None, # <--- NEW ARGUMENT
                               default_start_date='2020-01-01', 
                               default_lookback=126, 
                               default_holding=63,
                               default_strategy='Sharpe (ATR)', 
                               default_rank_start=1, 
                               default_rank_end=10,
                               default_benchmark_ticker='SPY', 
                               master_calendar_ticker='XOM', 
                               quality_thresholds=None, 
                               debug=False):
    
    # Pass both optimizations to the Engine
    engine = AlphaEngine(
        df_ohlcv, 
        features_df=precomputed_features, 
        df_close_wide=precomputed_close, # <--- PASS IT HERE
        master_ticker=master_calendar_ticker
    )
    results_container = [None]
    debug_container = [None]

    if quality_thresholds is None:
        quality_thresholds = {
            'min_median_dollar_volume': 100_000, 
            'min_liquidity_percentile': 0.50,    
            'max_stale_pct': 0.05, 
            'max_same_vol_count': 10
        }

    # --- 1. Define Widgets (Fixed Truncation) ---
    mode_selector = widgets.RadioButtons(options=['Ranking', 'Manual List'], value='Ranking', description='Mode:', layout={'width': 'max-content'}, style={'description_width': 'initial'})
    
    # Timeline Widgets - Added style={'description_width': 'initial'} and increased width slightly
    lookback_input = widgets.IntText(
        value=default_lookback, 
        description='Lookback (Days):', 
        layout={'width': '200px'}, 
        style={'description_width': 'initial'}
    )
    
    decision_date_picker = widgets.DatePicker(
        description='Decision Date:', 
        value=pd.to_datetime(default_start_date), 
        layout={'width': 'auto'},
        style={'description_width': 'initial'}
    )
    
    holding_input = widgets.IntText(
        value=default_holding, 
        description='Holding (Days):', 
        layout={'width': '200px'},
        style={'description_width': 'initial'}
    )
    
    # Strategy Widgets
    strategy_dropdown = widgets.Dropdown(
        options=list(METRIC_REGISTRY.keys()), 
        value=default_strategy, 
        description='Strategy:', 
        layout={'width': '220px'},
        style={'description_width': 'initial'}
    )
    
    benchmark_input = widgets.Text(
        value=default_benchmark_ticker, 
        description='Benchmark:', 
        placeholder='Enter Ticker', 
        layout={'width': '180px'},
        style={'description_width': 'initial'}
    )
    
    # Ranking Widgets
    rank_start_input = widgets.IntText(value=default_rank_start, description='Rank Start:', layout={'width': '150px'}, style={'description_width': 'initial'})
    rank_end_input = widgets.IntText(value=default_rank_end, description='Rank End:', layout={'width': '150px'}, style={'description_width': 'initial'})
    
    manual_tickers_input = widgets.Textarea(value='', placeholder='Enter tickers...', description='Manual Tickers:', layout={'width': '400px', 'height': '80px'}, style={'description_width': 'initial'})
    
    update_button = widgets.Button(description="Run Simulation", button_style='primary')
    ticker_list_output = widgets.Output()



    # --- 2. Create Layouts (Timeline Flow) ---
    timeline_box = widgets.HBox([lookback_input, decision_date_picker, holding_input], layout=widgets.Layout(justify_content='space-between', border='1px solid #ddd', padding='10px', margin='5px'))
    strategy_box = widgets.HBox([strategy_dropdown, benchmark_input])
    ranking_box = widgets.HBox([rank_start_input, rank_end_input])
    
    # Dynamic Visibility
    def on_mode_change(c):
        ranking_box.layout.display = 'flex' if c['new'] == 'Ranking' else 'none'
        manual_tickers_input.layout.display = 'none' if c['new'] == 'Ranking' else 'flex'
        strategy_dropdown.disabled = (c['new'] == 'Manual List')
    
    mode_selector.observe(on_mode_change, names='value')
    on_mode_change({'new': mode_selector.value})

    ui = widgets.VBox([
        widgets.HTML("<b>1. Timeline Configuration:</b> (Past <--- Decision ---> Future)"),
        timeline_box,
        widgets.HTML("<b>2. Strategy Settings:</b>"),
        widgets.HBox([mode_selector, strategy_box]),
        ranking_box,
        manual_tickers_input,
        widgets.HTML("<hr>"),
        update_button,
        ticker_list_output
    ], layout=widgets.Layout(margin='10px 0 20px 0'))
    
    fig = go.FigureWidget()
    fig.update_layout(title='Event-Driven Walk-Forward Analysis', height=600, template="plotly_white", hovermode='x unified')
    for i in range(50): fig.add_trace(go.Scatter(visible=False, line=dict(width=2)))
    fig.add_trace(go.Scatter(name='Benchmark', visible=True, line=dict(color='black', width=3, dash='dash')))
    fig.add_trace(go.Scatter(name='Group Portfolio', visible=True, line=dict(color='green', width=3)))

    def update_plot(b):
        ticker_list_output.clear_output()
        manual_list = [t.strip().upper() for t in manual_tickers_input.value.split(',') if t.strip()]
        decision_date_raw = pd.to_datetime(decision_date_picker.value)
        
        # MAPPING UI VALUES TO ENGINE INPUTS
        # start_date -> Decision Date
        # calc_period -> Lookback
        # fwd_period -> Holding
        inputs = EngineInput(
            mode=mode_selector.value,
            start_date=decision_date_raw, 
            calc_period=lookback_input.value,
            fwd_period=holding_input.value,
            metric=strategy_dropdown.value,
            benchmark_ticker=benchmark_input.value.strip().upper(),
            rank_start=rank_start_input.value,
            rank_end=rank_end_input.value,
            quality_thresholds=quality_thresholds,
            manual_tickers=manual_list,
            debug=debug
        )
        
        with ticker_list_output:
            res = engine.run(inputs)
            results_container[0] = res
            debug_container[0] = res.debug_data
            if res.error_msg: print(f"❌ {res.error_msg}"); return

            with fig.batch_update():
                cols = res.normalized_plot_data.columns.tolist()
                for i in range(50):
                    if i < len(cols): fig.data[i].update(x=res.normalized_plot_data.index, y=res.normalized_plot_data[cols[i]], name=cols[i], visible=True)
                    else: fig.data[i].visible = False
                
                fig.data[50].update(x=res.benchmark_series.index, y=res.benchmark_series.values, name=f"Benchmark ({inputs.benchmark_ticker})", visible=not res.benchmark_series.empty)
                fig.data[51].update(x=res.portfolio_series.index, y=res.portfolio_series.values, visible=True)
                
                # Draw Line at DECISION DATE (calc_end_date)
                fig.layout.shapes = [dict(type="line", x0=res.calc_end_date, y0=0, x1=res.calc_end_date, y1=1, xref='x', yref='paper', line=dict(color="red", width=2, dash="dash"))]
                fig.layout.annotations = [dict(x=res.calc_end_date, y=0.05, xref="x", yref="paper", text="DECISION", showarrow=False, bgcolor="red", font=dict(color="white"))]

            req_date = inputs.start_date.date()
            act_date = res.calc_end_date.date() # Decision date used
            
            # --- LIQUIDITY AUDIT PRINT ---
            if inputs.mode == 'Ranking' and res.debug_data and 'audit_liquidity' in res.debug_data:
                audit = res.debug_data['audit_liquidity']
                if audit:
                    pct_str = f"{audit.get('percentile_setting', 0)*100:.0f}%"
                    cut_val = audit.get('final_cutoff_usd', 0)
                    print("-" * 60)
                    print(f"🔍 LIQUIDITY CHECK (On Decision Date: {act_date})")
                    print(f"   Universe Size: {audit.get('total_tickers_available')} tickers")
                    print(f"   Filtering: Top {pct_str} of Market")
                    print(f"   Calculated Cutoff: ${cut_val:,.0f} / day")
                    print(f"   Tickers Remaining: {audit.get('tickers_passed')}")
                    print("-" * 60)
            
            print(f"Simulation Timeline: {res.start_date.date()} (Start) <--- [ {act_date} ] ---> {res.viz_end_date.date()} (End)")

            if inputs.mode == 'Ranking':
                print(f"Ranked Tickers ({len(res.tickers)}):")
                # Slice list into chunks of 10
                for i in range(0, len(res.tickers), 10):
                    print(", ".join(res.tickers[i:i+10]))
            else:
                print("Manual Portfolio Tickers:")
                print(", ".join(res.tickers))            

            m = res.perf_metrics
            rows = [
                {'Metric': 'Group Portfolio Gain', 'Full': m.get('full_p_gain'), 'Lookback': m.get('calc_p_gain'), 'Holding': m.get('fwd_p_gain')},
                {'Metric': f'Benchmark ({inputs.benchmark_ticker}) Gain', 'Full': m.get('full_b_gain'), 'Lookback': m.get('calc_b_gain'), 'Holding': m.get('fwd_b_gain')},
                {'Metric': '== Gain Delta', 'Full': m.get('full_p_gain',0)-m.get('full_b_gain',0), 'Lookback': m.get('calc_p_gain',0)-m.get('calc_b_gain',0), 'Holding': m.get('fwd_p_gain',0)-m.get('fwd_b_gain',0)},
                
                {'Metric': 'Group Sharpe', 'Full': m.get('full_p_sharpe'), 'Lookback': m.get('calc_p_sharpe'), 'Holding': m.get('fwd_p_sharpe')},
                {'Metric': f'Benchmark Sharpe', 'Full': m.get('full_b_sharpe'), 'Lookback': m.get('calc_b_sharpe'), 'Holding': m.get('fwd_b_sharpe')},
                {'Metric': '== Sharpe Delta', 'Full': m.get('full_p_sharpe',0)-m.get('full_b_sharpe',0), 'Lookback': m.get('calc_p_sharpe',0)-m.get('calc_b_sharpe',0), 'Holding': m.get('fwd_p_sharpe',0)-m.get('fwd_b_sharpe',0)},

                {'Metric': 'Group Sharpe (ATR)', 'Full': m.get('full_p_sharpe_atr'), 'Lookback': m.get('calc_p_sharpe_atr'), 'Holding': m.get('fwd_p_sharpe_atr')},
                {'Metric': f'Benchmark Sharpe (ATR)', 'Full': m.get('full_b_sharpe_atr'), 'Lookback': m.get('calc_b_sharpe_atr'), 'Holding': m.get('fwd_b_sharpe_atr')},
                {'Metric': '== Sharpe (ATR) Delta', 'Full': m.get('full_p_sharpe_atr',0)-m.get('full_b_sharpe_atr',0), 'Lookback': m.get('calc_p_sharpe_atr',0)-m.get('calc_b_sharpe_atr',0), 'Holding': m.get('fwd_p_sharpe_atr',0)-m.get('fwd_b_sharpe_atr',0)}
            ]
            display(pd.DataFrame(rows).set_index('Metric').style.format("{:+.4f}", na_rep="N/A"))

    update_button.on_click(update_plot)
    update_plot(None)
    display(ui, fig)
    return results_container, debug_container

# ==============================================================================
# SECTION F: UTILITIES
# ==============================================================================

def print_nested(d, indent=0, width=4):
    """Pretty-print any nested dict/list/tuple combination."""
    spacing = ' ' * indent
    if isinstance(d, dict):
        for k, v in d.items():
            print(f'{spacing}{k}:')
            print_nested(v, indent + width, width)
    elif isinstance(d, (list, tuple)):
        for item in d:
            print_nested(item, indent, width)
    else:
        print(f'{spacing}{d}')

def get_ticker_OHLCV(
    df_ohlcv: pd.DataFrame,
    tickers: Union[str, List[str]],
    date_start: str,
    date_end: str,
    return_format: str = "dataframe",
    verbose: bool = True
) -> Union[pd.DataFrame, dict]:
    """
    Get OHLCV data for specified tickers within a date range.
    
    Parameters
    ----------
    df_ohlcv : pd.DataFrame
        DataFrame with MultiIndex of (ticker, date) and OHLCV columns
    tickers : str or list of str
        Ticker symbol(s) to retrieve
    date_start : str
        Start date in 'YYYY-MM-DD' format
    date_end : str
        End date in 'YYYY-MM-DD' format
    return_format : str, optional
        Format to return data in. Options: 
        - 'dataframe': Single DataFrame with MultiIndex (default)
        - 'dict': Dictionary with tickers as keys and DataFrames as values
        - 'separate': List of separate DataFrames for each ticker
    verbose : bool, optional
        Whether to print summary information (default: True)
    
    Returns
    -------
    Union[pd.DataFrame, dict, list]
        Filtered OHLCV data in specified format
    
    Raises
    ------
    ValueError
        If input parameters are invalid
    KeyError
        If tickers not found in DataFrame
    
    Examples
    --------
    >>> # Get data for single ticker
    >>> vlo_data = get_ticker_OHLCV(df_ohlcv, 'VLO', '2025-08-13', '2025-09-04')
    
    >>> # Get data for multiple tickers
    >>> multi_data = get_ticker_OHLCV(df_ohlcv, ['VLO', 'JPST'], '2025-08-13', '2025-09-04')
    
    >>> # Get data as dictionary
    >>> data_dict = get_ticker_OHLCV(df_ohlcv, ['VLO', 'JPST'], '2025-08-13', 
    ...                              '2025-09-04', return_format='dict')
    """
    
    # Input validation
    if not isinstance(df_ohlcv, pd.DataFrame):
        raise TypeError("df_ohlcv must be a pandas DataFrame")
    
    if not isinstance(df_ohlcv.index, pd.MultiIndex):
        raise ValueError("DataFrame must have MultiIndex of (ticker, date)")
    
    if len(df_ohlcv.index.levels) != 2:
        raise ValueError("MultiIndex must have exactly 2 levels: (ticker, date)")
    
    # Convert single ticker to list for consistent processing
    if isinstance(tickers, str):
        tickers = [tickers]
    elif not isinstance(tickers, list):
        raise TypeError("tickers must be a string or list of strings")
    
    # Convert dates to Timestamps
    try:
        start_date = pd.Timestamp(date_start)
        end_date = pd.Timestamp(date_end)
    except ValueError as e:
        raise ValueError(f"Invalid date format. Use 'YYYY-MM-DD': {e}")
    
    if start_date > end_date:
        raise ValueError("date_start must be before or equal to date_end")
    
    # Check if tickers exist in the DataFrame
    available_tickers = df_ohlcv.index.get_level_values(0).unique()
    missing_tickers = [t for t in tickers if t not in available_tickers]
    
    if missing_tickers:
        raise KeyError(f"Ticker(s) not found in DataFrame: {missing_tickers}")
    
    # Filter the data using MultiIndex slicing
    try:
        filtered_data = df_ohlcv.loc[(tickers, slice(date_start, date_end)), :]
    except Exception as e:
        raise ValueError(f"Error filtering data: {e}")
    
    # Handle empty results
    if filtered_data.empty:
        if verbose:
            print(f"No data found for tickers {tickers} in date range {date_start} to {date_end}")
        return filtered_data
    
    # Print summary if verbose
    if verbose:
        print(f"Data retrieved for {len(tickers)} ticker(s) from {date_start} to {date_end}")
        print(f"Total rows: {len(filtered_data)}")
        print(f"Date range in data: {filtered_data.index.get_level_values(1).min()} to "
              f"{filtered_data.index.get_level_values(1).max()}")
        
        # Print ticker-specific counts
        ticker_counts = filtered_data.index.get_level_values(0).value_counts()
        for ticker in tickers:
            count = ticker_counts.get(ticker, 0)
            if count > 0:
                print(f"  {ticker}: {count} rows")
            else:
                print(f"  {ticker}: No data in range")
    
    # Return in requested format
    if return_format == "dict":
        result = {}
        for ticker in tickers:
            try:
                result[ticker] = filtered_data.xs(ticker, level=0).loc[date_start:date_end]
            except KeyError:
                result[ticker] = pd.DataFrame()
        return result
    
    elif return_format == "separate":
        result = []
        for ticker in tickers:
            try:
                result.append(filtered_data.xs(ticker, level=0).loc[date_start:date_end])
            except KeyError:
                result.append(pd.DataFrame())
        return result
    
    elif return_format == "dataframe":
        return filtered_data
    
    else:
        raise ValueError(f"Invalid return_format: {return_format}. "
                         f"Must be 'dataframe', 'dict', or 'separate'")

def get_ticker_features(
    features_df: pd.DataFrame,
    tickers: Union[str, List[str]],
    date_start: str,
    date_end: str,
    return_format: str = "dataframe",
    verbose: bool = True
) -> Union[pd.DataFrame, dict]:
    """
    Get features data for specified tickers within a date range.
    
    Parameters
    ----------
    features_df : pd.DataFrame
        DataFrame with MultiIndex of (ticker, date) and feature columns
    tickers : str or list of str
        Ticker symbol(s) to retrieve
    date_start : str
        Start date in 'YYYY-MM-DD' format
    date_end : str
        End date in 'YYYY-MM-DD' format
    return_format : str, optional
        Format to return data in. Options: 
        - 'dataframe': Single DataFrame with MultiIndex (default)
        - 'dict': Dictionary with tickers as keys and DataFrames as values
        - 'separate': List of separate DataFrames for each ticker
    verbose : bool, optional
        Whether to print summary information (default: True)
    
    Returns
    -------
    Union[pd.DataFrame, dict, list]
        Filtered features data in specified format
    """
    # Convert single ticker to list for consistent processing
    if isinstance(tickers, str):
        tickers = [tickers]
    
    # Filter the data using MultiIndex slicing
    try:
        filtered_data = features_df.loc[(tickers, slice(date_start, date_end)), :]
    except Exception as e:
        if verbose:
            print(f"Error filtering data: {e}")
        return pd.DataFrame() if return_format == "dataframe" else {}
    
    # Handle empty results
    if filtered_data.empty:
        if verbose:
            print(f"No data found for tickers {tickers} in date range {date_start} to {date_end}")
        return filtered_data
    
    # Print summary if verbose
    if verbose:
        print(f"Features data retrieved for {len(tickers)} ticker(s) from {date_start} to {date_end}")
        print(f"Total rows: {len(filtered_data)}")
        print(f"Date range in data: {filtered_data.index.get_level_values(1).min()} to "
              f"{filtered_data.index.get_level_values(1).max()}")
        print(f"Available features: {', '.join(filtered_data.columns.tolist())}")
        
        # Print ticker-specific counts
        ticker_counts = filtered_data.index.get_level_values(0).value_counts()
        for ticker in tickers:
            count = ticker_counts.get(ticker, 0)
            if count > 0:
                print(f"  {ticker}: {count} rows")
            else:
                print(f"  {ticker}: No data in range")
    
    # Return in requested format
    if return_format == "dict":
        result = {}
        for ticker in tickers:
            try:
                result[ticker] = filtered_data.xs(ticker, level=0).loc[date_start:date_end]
            except KeyError:
                result[ticker] = pd.DataFrame()
        return result
    
    elif return_format == "separate":
        result = []
        for ticker in tickers:
            try:
                result.append(filtered_data.xs(ticker, level=0).loc[date_start:date_end])
            except KeyError:
                result.append(pd.DataFrame())
        return result
    
    elif return_format == "dataframe":
        return filtered_data
    
    else:
        raise ValueError(f"Invalid return_format: {return_format}. "
                         f"Must be 'dataframe', 'dict', or 'separate'")

def create_combined_dict(
    df_ohlcv: pd.DataFrame,
    features_df: pd.DataFrame,
    tickers: Union[str, List[str]],
    date_start: str,
    date_end: str,
    verbose: bool = True
) -> dict:
    """
    Create a combined dictionary with both OHLCV and features data for each ticker.
    
    Parameters:
    -----------
    df_ohlcv : pd.DataFrame
        DataFrame with OHLCV data (MultiIndex: ticker, date)
    features_df : pd.DataFrame
        DataFrame with features data (MultiIndex: ticker, date)
    tickers : str or list of str
        Ticker symbol(s) to retrieve
    date_start : str
        Start date in 'YYYY-MM-DD' format
    date_end : str
        End date in 'YYYY-MM-DD' format
    verbose : bool, optional
        Whether to print progress information (default: True)
    
    Returns:
    --------
    dict
        Dictionary with tickers as keys and combined DataFrames (OHLCV + features) as values
    """
    # Convert single ticker to list
    if isinstance(tickers, str):
        tickers = [tickers]
    
    if verbose:
        print(f"Creating combined dictionary for {len(tickers)} ticker(s)")
        print(f"Date range: {date_start} to {date_end}")
        print("=" * 60)
    
    # Get OHLCV data as dictionary
    ohlcv_dict = get_ticker_OHLCV(
        df_ohlcv, tickers, date_start, date_end, 
        return_format='dict', verbose=verbose
    )
    
    # Get features data as dictionary
    features_dict = get_ticker_features(
        features_df, tickers, date_start, date_end,
        return_format='dict', verbose=verbose
    )
    
    # Create combined_dict
    combined_dict = {}
    
    for ticker in tickers:
        if verbose:
            print(f"\nProcessing {ticker}...")
        
        # Check if ticker exists in both dictionaries
        if ticker in ohlcv_dict and ticker in features_dict:
            ohlcv_data = ohlcv_dict[ticker]
            features_data = features_dict[ticker]
            
            # Check if both dataframes have data
            if not ohlcv_data.empty and not features_data.empty:
                # Combine OHLCV and features data
                # Note: Both dataframes have the same index (dates), so we can concatenate
                combined_df = pd.concat([ohlcv_data, features_data], axis=1)
                
                # Ensure proper index naming
                combined_df.index.name = 'Date'
                
                # Store in combined_dict
                combined_dict[ticker] = combined_df
                
                if verbose:
                    print(f"  ✓ Successfully combined data")
                    print(f"  OHLCV shape: {ohlcv_data.shape}")
                    print(f"  Features shape: {features_data.shape}")
                    print(f"  Combined shape: {combined_df.shape}")
                    print(f"  Date range: {combined_df.index.min()} to {combined_df.index.max()}")
            else:
                if verbose:
                    print(f"  ✗ Cannot combine: One or both dataframes are empty")
                    print(f"    OHLCV empty: {ohlcv_data.empty}")
                    print(f"    Features empty: {features_data.empty}")
                combined_dict[ticker] = pd.DataFrame()
        else:
            if verbose:
                print(f"  ✗ Ticker not found in both dictionaries")
                if ticker not in ohlcv_dict:
                    print(f"    Not in OHLCV data")
                if ticker not in features_dict:
                    print(f"    Not in features data")
            combined_dict[ticker] = pd.DataFrame()
    
    # Print summary
    if verbose:
        print("\n" + "=" * 60)
        print("SUMMARY")
        print("=" * 60)
        print(f"Total tickers processed: {len(tickers)}")
        
        tickers_with_data = [ticker for ticker, df in combined_dict.items() if not df.empty]
        print(f"Tickers with combined data: {len(tickers_with_data)}")
        
        if tickers_with_data:
            print("\nTicker details:")
            for ticker in tickers_with_data:
                df = combined_dict[ticker]
                print(f"  {ticker}: {df.shape} - {df.index.min()} to {df.index.max()}")
                print(f"    Columns: {len(df.columns)}")
        
        empty_tickers = [ticker for ticker, df in combined_dict.items() if df.empty]
        if empty_tickers:
            print(f"\nTickers with no data: {', '.join(empty_tickers)}")
    
    return combined_dict


### Unit Test for Generated Features by Function generate_features

In [2]:
def test_true_range_calculation():
    """Test TR = max(High-Low, |High-PrevClose|, |Low-PrevClose|) using robust pandas testing"""
    print("Running test_true_range_calculation (Robust Version)...")
    
    # 1. SETUP: Create test data
    test_data = {
        'Adj Open': [100, 105, 95, 98, 102],
        'Adj High': [105, 108, 97, 102, 105],
        'Adj Low': [95, 103, 93, 100, 98],
        'Adj Close': [100, 106, 96, 99, 103],
        'Volume': [1000, 1200, 800, 900, 1100]
    }
    
    index = pd.MultiIndex.from_tuples(
        [
            ('TEST', pd.Timestamp('2024-01-01')),
            ('TEST', pd.Timestamp('2024-01-02')),
            ('TEST', pd.Timestamp('2024-01-03')),
            ('TEST', pd.Timestamp('2024-01-04')),
            ('TEST', pd.Timestamp('2024-01-05')),
        ],
        names=['Ticker', 'Date']
    )
    
    df_test = pd.DataFrame(test_data, index=index)
    print(f'test_true_range_df input:\n{df_test}\n')
    
    # 2. EXECUTION: Run function
    result = generate_features(df_test, quality_window=5, quality_min_periods=2)
    
    # 3. ASSERTION: Define EXACT expected series
    # Day 1: NaN (No prev close)
    # Day 2: 8.0 (PrevClose=100, High=108, Low=103. |108-100|=8)
    # Day 3: 13.0 (PrevClose=106, High=97, Low=93. |93-106|=13)
    # Day 4: 6.0 (PrevClose=96, High=102, Low=100. |102-96|=6)
    # Day 5: 7.0 (PrevClose=99, High=105, Low=98. High-Low=7)
    expected_tr_values = [np.nan, 8.0, 13.0, 6.0, 7.0]
    
    expected_series = pd.Series(
        expected_tr_values, 
        index=result.index, 
        name='TR',
        dtype='float64'
    )
    
    try:
        # Check Series Equality
        # check_exact=False allows for minor floating point differences (rtol=1e-4)
        assert_series_equal(result['TR'], expected_series, check_exact=False, rtol=1e-4)
        
        print("✅ PD Testing Assertion Passed! All TR values match expected logic.")
        return True
        
    except AssertionError as e:
        print(f"❌ PD Testing Assertion Failed: {e}")
        
        # Helper output to see what went wrong
        print("\nDetailed Comparison (Actual vs Expected):")
        comparison = pd.concat([result['TR'], expected_series], axis=1, keys=['Actual_TR', 'Expected_TR'])
        comparison['Diff'] = comparison['Actual_TR'] - comparison['Expected_TR']
        print(comparison)
        
        return False

def test_atr_calculation():
    """Test ATR = EWMA of TR with alpha=1/period"""
    print("\n" + "="*50)
    print("Running test_atr_calculation...")
    
    # Test data with 5 days
    test_data = {
        'Adj Open': [100, 102, 103, 110, 108],
        'Adj High': [101, 103, 103, 112, 110],
        'Adj Low': [99, 101, 103, 108, 107],
        'Adj Close': [100, 102, 103, 111, 109],
        'Volume': [1000, 1000, 1000, 1000, 1000]  # All non-zero for simplicity
    }
    
    index = pd.MultiIndex.from_tuples(
        [('TEST', pd.Timestamp(f'2024-01-{i:02d}')) for i in range(1, 6)],
        names=['Ticker', 'Date']
    )
    
    df_test = pd.DataFrame(test_data, index=index)

    print(f'test_true_range_df:\n{df_test}\n')

    result = generate_features(df_test, atr_period=14)
    
    print("\nATR Calculation Results:")
    print(result[['TR', 'ATR', 'ATRP']])
    
    # Manual calculation from our earlier example
    # CORRECTED EXPECTED VALUES WITH MORE PRECISION
    expected_atr = {
        '2024-01-02': 3.0,
        '2024-01-03': 40/14,  # ≈ 2.857142857142857
        '2024-01-04': 646/196,  # ≈ 3.2959183673469388
        '2024-01-05': 9182/2744,  # ≈ 3.3462099125364433
    }

    all_passed = True
    for date_str, expected in expected_atr.items():
        actual = result.loc[('TEST', pd.Timestamp(date_str)), 'ATR']
        if abs(actual - expected) < 0.0001:
            print(f"✓ {date_str} ATR: {actual:.6f} ≈ {expected:.6f}")
        else:
            print(f"✗ {date_str} ATR: {actual:.6f} != {expected:.6f}")
            all_passed = False
    
    if all_passed:
        print("\n✅ All ATR tests passed!")
    else:
        print("\n❌ Some ATR tests failed!")
    
    return all_passed

def test_is_stale_calculation():
    """Test IsStale = 1 when Volume=0 OR High=Low"""
    print("\n" + "="*50)
    print("Running test_is_stale_calculation...")
    
    test_data = {
        'Adj Open': [100, 102, 103, 104],
        'Adj High': [101, 103, 103, 105],  # Day 3: High=Low
        'Adj Low': [99, 101, 103, 104],
        'Adj Close': [100, 102, 103, 105],
        'Volume': [1000, 0, 500, 1000]  # Day 2: Volume=0
    }
    
    index = pd.MultiIndex.from_tuples(
        [('TEST', pd.Timestamp(f'2024-01-{i:02d}')) for i in range(1, 5)],
        names=['Ticker', 'Date']
    )
    
    df_test = pd.DataFrame(test_data, index=index)

    print(f'test_is_stale_df:\n{df_test}\n')

    # Create IsStale manually to verify
    is_stale_manual = np.where(
        (df_test['Volume'] == 0) | (df_test['Adj High'] == df_test['Adj Low']),
        1, 0
    )
    
    print("\n📊 Manual IsStale Calculation:")
    print("=" * 50)
    print("IsStale = 1 if EITHER condition is true:")
    print("  1. Volume == 0")
    print("  2. Adj High == Adj Low (no price movement)")
    print("Otherwise, IsStale = 0")
    print("=" * 50)
    
    # Create a temporary DataFrame to display the calculation clearly
    manual_calc_df = df_test.copy()
    manual_calc_df['IsStale_Manual'] = is_stale_manual
    manual_calc_df['Volume==0'] = manual_calc_df['Volume'] == 0
    manual_calc_df['High==Low'] = manual_calc_df['Adj High'] == manual_calc_df['Adj Low']
    
    print("\nCalculation details:")
    for idx, row in manual_calc_df.iterrows():
        ticker_date = f"{idx[0]}, {idx[1].strftime('%Y-%m-%d')}"
        conditions = []
        if row['Volume==0']:
            conditions.append("Volume=0")
        if row['High==Low']:
            conditions.append("High=Low")
        
        condition_str = " OR ".join(conditions) if conditions else "None (both False)"
        result = row['IsStale_Manual']
        
        print(f"  {ticker_date}:")
        print(f"    Volume={row['Volume']}, High={row['Adj High']}, Low={row['Adj Low']}")
        print(f"    Conditions met: {condition_str}")
        print(f"    → IsStale = {result}")
        print()

    expected = [0, 1, 1, 0]  # Day 1: normal, Day 2: vol=0, Day 3: high=low, Day 4: normal
    
    print(f"\nManual IsStale calculation: {is_stale_manual}")
    print(f"Expected: {expected}")
    
    if list(is_stale_manual) == expected:
        print("✓ IsStale calculation logic is correct")
        return True
    else:
        print(f"✗ IsStale calculation failed. Got {is_stale_manual}, expected {expected}")
        return False

def test_multiple_tickers():
    """Test that calculations don't mix data between tickers"""
    print("\n" + "="*50)
    print("Running test_multiple_tickers...")
    
    test_data = {
        'Adj Open': [100, 102, 50, 51],
        'Adj High': [101, 103, 52, 53],
        'Adj Low': [99, 101, 48, 49],
        'Adj Close': [100, 102, 49, 52],
        'Volume': [1000, 1000, 2000, 2000]
    }
    
    index = pd.MultiIndex.from_tuples([
        ('A', pd.Timestamp('2024-01-01')),
        ('A', pd.Timestamp('2024-01-02')),
        ('B', pd.Timestamp('2024-01-01')),
        ('B', pd.Timestamp('2024-01-02')),
    ], names=['Ticker', 'Date'])
    
    df_test = pd.DataFrame(test_data, index=index)

    print(f'test_multiple_tickers_df:\n{df_test}\n')

    result = generate_features(df_test)
    
    print("\nMultiple Ticker Results:")
    print(result[['TR', 'ATR']])
    
    # Ticker A day 2 TR should use A day 1 close, not B day 1 close
    tr_a2 = result.loc[('A', '2024-01-02'), 'TR']
    expected_a2 = 3.0  # max(103-101=2, |103-100|=3, |101-100|=1) = 3
    
    tr_b2 = result.loc[('B', '2024-01-02'), 'TR']
    expected_b2 = 4.0  # max(53-49=4, |53-49|=4, |49-49|=0) = 4
    
    tests_passed = 0
    total_tests = 2
    
    if abs(tr_a2 - expected_a2) < 0.0001:
        print(f"✓ Ticker A TR: {tr_a2} (expected {expected_a2})")
        tests_passed += 1
    else:
        print(f"✗ Ticker A TR: {tr_a2} != {expected_a2}")
    
    if abs(tr_b2 - expected_b2) < 0.0001:
        print(f"✓ Ticker B TR: {tr_b2} (expected {expected_b2})")
        tests_passed += 1
    else:
        print(f"✗ Ticker B TR: {tr_b2} != {expected_b2}")
    
    if tests_passed == total_tests:
        print("✅ Ticker separation test passed!")
        return True
    else:
        print(f"❌ Ticker separation test failed: {tests_passed}/{total_tests} passed")
        return False

def test_edge_cases():
    """Test edge cases like zero price, single row, etc."""
    print("\n" + "="*50)
    print("Running test_edge_cases...")
    
    all_passed = True
    
    # Test 1: Very low price (penny stock)
    print("\n1. Testing penny stock with low price...")
    test_data = {
        'Adj Open': [0.10, 0.11],
        'Adj High': [0.10, 0.11],
        'Adj Low': [0.10, 0.11],
        'Adj Close': [0.10, 0.11],
        'Volume': [1000, 1000]
    }
    
    index = pd.MultiIndex.from_tuples([
        ('PENNY', pd.Timestamp('2024-01-01')),
        ('PENNY', pd.Timestamp('2024-01-02')),
    ], names=['Ticker', 'Date'])
    
    df_penny = pd.DataFrame(test_data, index=index)

    print(f'df_penny_stock:\n{df_penny}\n')

    result = generate_features(df_penny)
    
    # Check ATRP is reasonable (not inf/nan)
    atrp_val = result.loc[('PENNY', '2024-01-02'), 'ATRP']
    if pd.isna(atrp_val) or np.isinf(atrp_val):
        print(f"✗ Penny stock ATRP is {atrp_val} (should be finite)")
        all_passed = False
    else:
        print(f"✓ Penny stock ATRP is {atrp_val:.4f}")
    
    # Test 2: Single row
    print("\n2. Testing single row data...")
    test_data_single = {
        'Adj Open': [100],
        'Adj High': [101],
        'Adj Low': [99],
        'Adj Close': [100],
        'Volume': [1000]
    }
    
    index_single = pd.MultiIndex.from_tuples(
        [('SINGLE', pd.Timestamp('2024-01-01'))],
        names=['Ticker', 'Date']
    )
    
    df_single = pd.DataFrame(test_data_single, index=index_single)

    print(f'df_single:\n{df_single}\n')

    result_single = generate_features(df_single, quality_window=3, quality_min_periods=2)
    
    # TR should be NaN (no previous close)
    if pd.isna(result_single.loc[('SINGLE', '2024-01-01'), 'TR']):
        print("✓ Single row TR is NaN (correct)")
    else:
        print(f"✗ Single row TR should be NaN but got {result_single.loc[('SINGLE', '2024-01-01'), 'TR']}")
        all_passed = False
    
    # Rolling metrics should be NaN with min_periods=2
    if pd.isna(result_single.loc[('SINGLE', '2024-01-01'), 'RollingStalePct']):
        print("✓ Single row rolling metrics are NaN (correct - insufficient periods)")
    else:
        print(f"✗ Rolling metrics should be NaN but got {result_single.loc[('SINGLE', '2024-01-01'), 'RollingStalePct']}")
        all_passed = False
    
    if all_passed:
        print("\n✅ All edge case tests passed!")
    else:
        print("\n❌ Some edge case tests failed!")
    
    return all_passed

def test_zero_division_protection():
    """Test that Zero Price doesn't cause Inf values in ATRP"""
    print("\n" + "="*50)
    print("Running test_zero_division_protection...")

    test_data = {
        'Adj Open': [10, 10, 10],
        'Adj High': [12, 12, 12],
        'Adj Low': [8, 8, 8],
        'Adj Close': [10, 0, 10], # Day 2 Price is ZERO
        'Volume': [1000, 1000, 1000]
    }
    index = pd.MultiIndex.from_tuples(
        [('ZERO', pd.Timestamp(f'2024-01-0{i}')) for i in range(1, 4)],
        names=['Ticker', 'Date']
    )
    df_test = pd.DataFrame(test_data, index=index)

    print(f'test_zero_division_protection_df:\n{df_test}\n')

    # Run features
    result = generate_features(df_test, atr_period=2)
    
    # Check Day 2 ATRP
    atrp_val = result.loc[('ZERO', '2024-01-02'), 'ATRP']
    
    if pd.isna(atrp_val):
        print("✅ Zero Division Test Passed: ATRP is NaN when Close is 0.")
        return True
    elif np.isinf(atrp_val):
        print(f"❌ Zero Division Test Failed: ATRP is Infinite ({atrp_val}).")
        return False
    else:
        print(f"❌ Zero Division Test Failed: Unexpected value {atrp_val}")
        return False

def test_unsorted_input_handling():
    """Test that function handles unsorted dates correctly via sorting"""
    print("\n" + "="*50)    
    print("Running test_unsorted_input_handling...")
    
    # Data is out of order: Day 2, Day 1, Day 3
    index = pd.MultiIndex.from_tuples([
        ('A', pd.Timestamp('2024-01-02')), 
        ('A', pd.Timestamp('2024-01-01')), 
        ('A', pd.Timestamp('2024-01-03'))
    ], names=['Ticker', 'Date'])
    
    # Prices: 100 -> 105 -> 110
    # If processed in order given: 
    # 1. 105 (No prev)
    # 2. 100 (Prev is 105) -> Change -5
    # 3. 110 (Prev is 100) -> Change +10
    
    # If sorted correctly:
    # 1. 100 (No prev)
    # 2. 105 (Prev is 100) -> Change +5
    # 3. 110 (Prev is 105) -> Change +5
    
    test_data = {
        'Adj Open': [100, 100, 100],
        'Adj High': [105, 100, 110], 
        'Adj Low': [105, 100, 110],
        'Adj Close': [105, 100, 110], # 105, 100, 110
        'Volume': [100, 100, 100]
    }
    
    df_test = pd.DataFrame(test_data, index=index)

    print(f'test_unsorted_input_handling_df:\n{df_test}\n')

    # Run features
    result = generate_features(df_test)
    
    # Inspect 2024-01-02 (Should be Day 2 in sorted order)
    # Prev close (Jan 1) was 100. Current High 105. TR should be roughly 5.
    tr_day_2 = result.loc[('A', '2024-01-02'), 'TR']
    
    # If it wasn't sorted, Day 2 would be treated as the first row (TR=NaN) 
    # or compared against whatever came before it in memory.
    
    if pd.isna(tr_day_2):
         print("❌ Sorting Test Failed: Day 2 TR is NaN (likely treated as first row).")
         return False
    elif abs(tr_day_2 - 5.0) < 0.1:
         print("✅ Sorting Test Passed: Logic applied in correct chronological order.")
         return True
    else:
         print(f"❌ Sorting Test Failed: Day 2 TR is {tr_day_2}, expected ~5.0")
         return False

    """Test TR = max(High-Low, |High-PrevClose|, |Low-PrevClose|) using robust pandas testing"""
    print("Running test_true_range_calculation (Robust Version)...")
    
    # 1. SETUP: Create test data
    test_data = {
        'Adj Open': [100, 105, 95, 98, 102],
        'Adj High': [105, 108, 97, 102, 105],
        'Adj Low': [95, 103, 93, 100, 98],
        'Adj Close': [100, 106, 96, 99, 103],
        'Volume': [1000, 1200, 800, 900, 1100]
    }
    
    index = pd.MultiIndex.from_tuples(
        [
            ('TEST', pd.Timestamp('2024-01-01')),
            ('TEST', pd.Timestamp('2024-01-02')),
            ('TEST', pd.Timestamp('2024-01-03')),
            ('TEST', pd.Timestamp('2024-01-04')),
            ('TEST', pd.Timestamp('2024-01-05')),
        ],
        names=['Ticker', 'Date']
    )
    
    df_test = pd.DataFrame(test_data, index=index)
    print(f'test_true_range_df input:\n{df_test}\n')
    
    # 2. EXECUTION: Run function
    result = generate_features(df_test, quality_window=5, quality_min_periods=2)
    
    # 3. ASSERTION: Define EXACT expected series
    # Day 1: NaN (No prev close)
    # Day 2: 8.0 (PrevClose=100, High=108, Low=103. |108-100|=8)
    # Day 3: 13.0 (PrevClose=106, High=97, Low=93. |93-106|=13)
    # Day 4: 6.0 (PrevClose=96, High=102, Low=100. |102-96|=6)
    # Day 5: 7.0 (PrevClose=99, High=105, Low=98. High-Low=7)
    expected_tr_values = [np.nan, 8.0, 13.0, 6.0, 7.0]
    
    expected_series = pd.Series(
        expected_tr_values, 
        index=result.index, 
        name='TR',
        dtype='float64'
    )
    
    try:
        # Check Series Equality
        # check_exact=False allows for minor floating point differences (rtol=1e-4)
        assert_series_equal(result['TR'], expected_series, check_exact=False, rtol=1e-4)
        
        print("✅ PD Testing Assertion Passed! All TR values match expected logic.")
        return True
        
    except AssertionError as e:
        print(f"❌ PD Testing Assertion Failed: {e}")
        
        # Helper output to see what went wrong
        print("\nDetailed Comparison (Actual vs Expected):")
        comparison = pd.concat([result['TR'], expected_series], axis=1, keys=['Actual_TR', 'Expected_TR'])
        comparison['Diff'] = comparison['Actual_TR'] - comparison['Expected_TR']
        print(comparison)
        
        return False

def test_quality_rolling_features():
    """
    Test RollingStalePct, RollMedDollarVol, and RollingSameVolCount 
    verifying logic for Stale(Vol=0), Stale(H=L), SameVolume, and Median calculations.
    """
    print("\n" + "="*50)
    print("Running test_quality_rolling_features...")

    # 1. SETUP: Create specific test data
    # We set up 5 days to test a window of 4
    test_data = {
        # Day 1: Normal Base Day. $Vol = 10*100 = 1000.
        # Day 2: Same Volume as D1. $Vol = 10*100 = 1000.
        # Day 3: Stale (Volume=0). $Vol = 20*0 = 0.
        # Day 4: Stale (High=Low). $Vol = 20*50 = 1000.
        # Day 5: Normal High Vol. $Vol = 30*200 = 6000.
        
        'Adj Open':  [10,   10,   20,   20,   30],
        'Adj High':  [12,   12,   22,   20,   35], # Day 4 High=20
        'Adj Low':   [8,    8,    18,   20,   25], # Day 4 Low=20 (H=L)
        'Adj Close': [10,   10,   20,   20,   30],
        'Volume':    [100,  100,  0,    50,   200] # Day 2 same as D1, Day 3 is 0
    }

    index = pd.MultiIndex.from_tuples(
        [('TEST', pd.Timestamp(f'2024-01-0{i}')) for i in range(1, 6)],
        names=['Ticker', 'Date']
    )

    df_test = pd.DataFrame(test_data, index=index)
    print(f'Input Data:\n{df_test}')

    # 2. EXECUTION: Use window=4, min_periods=2 to capture partial rolling
    # We expect Day 1 to be NaN (count=1 < min_periods=2)
    result = generate_features(df_test, quality_window=4, quality_min_periods=2)
    
    # 3. VERIFICATION

    # --- A. Test RollingStalePct ---
    print("\n--- Testing RollingStalePct ---")
    # Logic:
    # Day 1: IsStale=0. Result=NaN (min_periods)
    # Day 2: IsStale=0. Window=[0,0]. Mean=0.0
    # Day 3: IsStale=1 (Vol=0). Window=[0,0,1]. Mean=1/3 (~0.333)
    # Day 4: IsStale=1 (High=Low). Window=[0,0,1,1]. Mean=2/4 = 0.5
    # Day 5: IsStale=0. Window=[0,1,1,0] (Day 1 drops off). Mean=2/4 = 0.5
    
    expected_stale = pd.Series(
        [np.nan, 0.0, 1/3, 0.5, 0.5],
        index=result.index, name='RollingStalePct'
    )
    
    try:
        assert_series_equal(result['RollingStalePct'], expected_stale, check_exact=False, rtol=1e-4)
        print("✅ RollingStalePct Passed")
    except AssertionError as e:
        print(f"❌ RollingStalePct Failed: {e}")
        return False

    # --- B. Test RollMedDollarVol ---
    print("\n--- Testing RollMedDollarVol ---")
    # Logic: $Vol = Close * Volume
    # D1: 1000. Result=NaN
    # D2: 1000. Window=[1000, 1000]. Median=1000
    # D3: 0.    Window=[1000, 1000, 0]. Sorted=[0, 1000, 1000]. Median=1000
    # D4: 1000. Window=[1000, 1000, 0, 1000]. Sorted=[0, 1000, 1000, 1000]. Median=(1000+1000)/2 = 1000
    # D5: 6000. Window=[1000, 0, 1000, 6000]. Sorted=[0, 1000, 1000, 6000]. Median=1000
    
    expected_dollar = pd.Series(
        [np.nan, 1000.0, 1000.0, 1000.0, 1000.0],
        index=result.index, name='RollMedDollarVol'
    )

    try:
        assert_series_equal(result['RollMedDollarVol'], expected_dollar, check_exact=False, rtol=1e-4)
        print("✅ RollMedDollarVol Passed")
    except AssertionError as e:
        print(f"❌ RollMedDollarVol Failed: {e}")
        return False

    # --- C. Test RollingSameVolCount ---
    print("\n--- Testing RollingSameVolCount ---")
    # Logic: HasSameVolume = (Volume == PrevVolume)
    # D1: NaN/0 (First row diff is NaN, astype(int) -> 0). Result=NaN (min_periods)
    # D2: 1 (100==100). Window=[0, 1]. Sum=1
    # D3: 0 (0!=100).   Window=[0, 1, 0]. Sum=1
    # D4: 0 (50!=0).    Window=[0, 1, 0, 0]. Sum=1
    # D5: 0 (200!=50).  Window=[1, 0, 0, 0] (D1 drops). Sum=1
    
    expected_same_vol = pd.Series(
        [np.nan, 1.0, 1.0, 1.0, 1.0],
        index=result.index, name='RollingSameVolCount'
    )

    try:
        assert_series_equal(result['RollingSameVolCount'], expected_same_vol, check_exact=False, rtol=1e-4)
        print("✅ RollingSameVolCount Passed")
    except AssertionError as e:
        print(f"❌ RollingSameVolCount Failed: {e}")
        return False

    print("\n🎉 All Rolling Quality Feature Tests Passed!")
    return True

def test_math_metrics():
    """Verifies calculate_gain and calculate_sharpe are mathematically precise."""
    print("\n" + "="*50)
    print("Running test_math_metrics...")
    all_passed = True

    # --- 1. Test calculate_gain ---
    # Case A: Price goes 100 -> 110 (10% gain)
    s_gain = pd.Series([100, 105, 110])
    res_gain = calculate_gain(s_gain)

    print(f's_gain:\n{s_gain}')
    print(f'res_gain:\n{res_gain}')    
    
    if abs(res_gain - 0.10) < 1e-9:
        print("✅ Gain Calc (Positive): Passed\n")
    else:
        print(f"❌ Gain Calc (Positive): Failed. Got {res_gain}\n")
        all_passed = False

    # Case B: Price goes 100 -> 90 (10% loss)
    s_loss = pd.Series([100, 95, 90])
    res_loss = calculate_gain(s_loss)

    print(f's_loss:\n{s_loss}')
    print(f'res_loss:\n{res_loss}')  

    if abs(res_loss - (-0.10)) < 1e-9:
        print("✅ Gain Calc (Negative): Passed\n")
    else:
        print(f"❌ Gain Calc (Negative): Failed. Got {res_loss}\n")
        all_passed = False

    # --- 2. Test calculate_sharpe ---
    # Case: Flat return (0 volatility) -> Should return 0.0 (handle divide by zero)
    s_flat = pd.Series([0.01, 0.01, 0.01]) 
    res_sharpe_flat = calculate_sharpe(s_flat)

    print(f's_flat:\n{s_flat}')
    print(f'res_sharpe_flat:\n{res_sharpe_flat}')  

    if res_sharpe_flat == 0.0:
        print("✅ Sharpe (Zero Volatility): Passed (Returns 0.0)\n")
    else:
        print(f"❌ Sharpe (Zero Volatility): Failed. Got {res_sharpe_flat}\n")
        all_passed = False
    
    return all_passed

def test_engine_lag_logic():
    """
    CRITICAL TEST: Verifies the '1-Day Lag' logic.
    If we make a decision on Day T, we cannot trade at Day T's close (Lookahead bias).
    We effectively enter at Day T+1 Close (in this implementation).
    
    Scenario:
    Day 1: 100
    Day 2: 100  <-- Decision Date (End of Calc Period)
    Day 3: 110  <-- T+1 (Price jumps 10%)
    Day 4: 121  <-- T+2 (Price jumps 10%)
    
    If 'Fwd Gain' includes Day 2->3 jump, it implies T+0 execution (Lookahead).
    If 'Fwd Gain' is Day 3->4, it implies T+1 execution (Realistic).
    """
    print("\n" + "="*50)
    print("\nRunning test_engine_lag_logic...")
    
    # 1. Setup Mock Data
    dates = pd.to_datetime(['2024-01-01', '2024-01-02', '2024-01-03', '2024-01-04'])
    # Prices: Flat, Flat, +10%, +10%
    prices = [100.0, 100.0, 110.0, 121.0] 
    
    df_test = pd.DataFrame({
        'Adj Close': prices,
        'Adj High': prices,
        'Adj Low': prices, 
        'Adj Open': prices,
        'Volume': [1000]*4
    }, index=pd.MultiIndex.from_product([['MOCK'], dates], names=['Ticker', 'Date']))

    print(f'df_test:\n{df_test}\n')

    # 2. Initialize Engine
    engine = AlphaEngine(df_test)
    
    # 3. Run Strategy
    # Decision Date = 2024-01-02.
    # We define Lookback=1 (uses Jan 1-2).
    # We define Forward=2 (uses Jan 3-4).
    inputs = EngineInput(
        mode='Manual List',
        start_date=pd.Timestamp('2024-01-02'),
        calc_period=1,
        fwd_period=2,
        metric='Price', # Irrelevant for manual
        benchmark_ticker='MOCK', # Compare against self
        manual_tickers=['MOCK']
    )
    
    res = engine.run(inputs)
    
    # 4. Analyze Results
    metrics = res.perf_metrics
    
    print(f"  Decision Date: {res.calc_end_date.date()}")
    print(f"  Full Series Gain: {metrics['full_p_gain']:.4f} (Should range Jan 1 -> Jan 4)")
    print(f"  Forward Gain: {metrics['fwd_p_gain']:.4f}")
    
    # --- VERIFICATION LOGIC ---
    
    # Full Gain: 100 -> 121 = 21% (0.2100)
    # Forward Logic in Engine:
    #   Slice starts at Decision Date (Jan 2).
    #   get_lagged_fwd(val, Jan 2) -> Slice [Jan 2, Jan 3, Jan 4] -> iloc[1:] -> [Jan 3, Jan 4].
    #   Calc Gain on [Jan 3 (110), Jan 4 (121)].
    #   Expected: (121 / 110) - 1 = 1.1 - 1 = 0.10 (10%).
    
    # If the engine was T+0 (Lookahead), it would use Jan 2 (100) -> Jan 4 (121) = 21%.
    
    expected_fwd_gain = 0.10
    
    if abs(metrics['fwd_p_gain'] - expected_fwd_gain) < 1e-4:
        print(f"✅ LAG VERIFIED: Forward Gain is {metrics['fwd_p_gain']:.2%}.")
        print("   The system correctly skips the jump immediately following the decision.")
        print("   It simulates entry at T+1 Close.")
        return True
    else:
        print(f"❌ LAG FAILURE: Forward Gain is {metrics['fwd_p_gain']:.2%}.")
        print(f"   Expected {expected_fwd_gain:.2%} (T+1 entry).")
        return False

def run_all_tests():
    """Run all tests and provide summary"""
    print("="*60)
    print("STARTING ALL TESTS FOR generate_features()")
    print("="*60)
    
    test_results = {}
    
    # Run each test
    test_results['TR Calculation'] = test_true_range_calculation()
    test_results['ATR Calculation'] = test_atr_calculation()
    test_results['IsStale Calculation'] = test_is_stale_calculation()
    test_results['Multiple Tickers'] = test_multiple_tickers()
    test_results['Edge Cases'] = test_edge_cases()
    test_results['Zero Division Protection'] = test_zero_division_protection()
    test_results['Unsorted Input Handling'] = test_unsorted_input_handling()    
    test_results['Quality Rolling Features'] = test_quality_rolling_features()   
    test_results['Math Metrics'] = test_math_metrics()    
    test_results['Engine Lag Logic'] = test_engine_lag_logic()   

    # Summary
    print("\n" + "="*60)
    print("TEST SUMMARY")
    print("="*60)
    
    passed = sum(test_results.values())
    total = len(test_results)
    
    for test_name, result in test_results.items():
        status = "✅ PASS" if result else "❌ FAIL"
        print(f"{status}: {test_name}")
    
    print("\n" + "="*60)
    print(f"TOTAL: {passed}/{total} tests passed ({passed/total*100:.1f}%)")
    print("="*60)
    
    if passed == total:
        print("\n🎉 ALL TESTS PASSED! Your feature calculations are working correctly.")
        return True
    else:
        print(f"\n⚠️  {total - passed} test(s) failed. Review the output above.")
        return False

run_all_tests()

STARTING ALL TESTS FOR generate_features()
Running test_true_range_calculation (Robust Version)...
test_true_range_df input:
                   Adj Open  Adj High  Adj Low  Adj Close  Volume
Ticker Date                                                      
TEST   2024-01-01       100       105       95        100    1000
       2024-01-02       105       108      103        106    1200
       2024-01-03        95        97       93         96     800
       2024-01-04        98       102      100         99     900
       2024-01-05       102       105       98        103    1100

✅ PD Testing Assertion Passed! All TR values match expected logic.

Running test_atr_calculation...
test_true_range_df:
                   Adj Open  Adj High  Adj Low  Adj Close  Volume
Ticker Date                                                      
TEST   2024-01-01       100       101       99        100    1000
       2024-01-02       102       103      101        102    1000
       2024-01-03       103 

True

In [3]:
data_path = r'c:\Users\ping\Files_win10\python\py311\stocks\data\df_OHLCV_stocks_etfs.parquet'
df_ohlcv = pd.read_parquet(data_path, engine='pyarrow')
print(f'df_ohlcv.info():\n{df_ohlcv.info()}')
df_ohlcv

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 9641787 entries, ('A', Timestamp('1999-11-18 00:00:00')) to ('ZWS', Timestamp('2025-12-11 00:00:00'))
Data columns (total 5 columns):
 #   Column     Dtype  
---  ------     -----  
 0   Adj Open   float64
 1   Adj High   float64
 2   Adj Low    float64
 3   Adj Close  float64
 4   Volume     int64  
dtypes: float64(4), int64(1)
memory usage: 405.3+ MB
df_ohlcv.info():
None


Adj Open  Adj High  Adj Low  Adj Close    Volume
Ticker Date                                                        
A      1999-11-18   27.2452   29.9398  23.9518    26.3470  74716417
       1999-11-19   25.7108   25.7482  23.8396    24.1764  18198344
       1999-11-22   24.7378   26.3470  23.9893    26.3470   7857766
       1999-11-23   25.4488   26.1225  23.9518    23.9518   7138322
       1999-11-24   24.0267   25.1120  23.9518    24.5881   5785608
...                     ...       ...      ...        ...       ...
ZWS    2025-12-05   46.8600   47.2700  46.4400    46.8600    886700
       2025-12-08   46.9400   47.2000  46.2900    46.3800    683900
       2025-12-09   46.2900   46.6500  45.8700    45.9300    605000
       2025-12-10   46.0100   47.1100  45.9400    46.8600   1248300
       2025-12-11   47.0600   47.8900  46.8800    47.5900    618204

[9641787 rows x 5 columns]

In [4]:
# Calculate features ONCE and store them in a variable
print("Calculating features... this might take few minutes...")
print("1. Calculating Features...")
my_features = generate_features(
    df_ohlcv=df_ohlcv, 
    atr_period=14,
    quality_window=252,
    quality_min_periods=126
)

print("2. Pivoting Price Matrix...")
# This is the line that takes 12 seconds, but now we only run it ONCE.
my_close_matrix = df_ohlcv['Adj Close'].unstack(level=0)

print("✅ Optimization Complete. Ready for UI.")

Calculating features... this might take few minutes...
1. Calculating Features...
2. Pivoting Price Matrix...
✅ Optimization Complete. Ready for UI.


In [5]:
print(f'my_features:\n{my_features}\n')
my_features.info()

my_features:
                       TR     ATR    ATRP   ROC_1   ROC_3   ROC_5  ROC_10  ROC_21  RollingStalePct  RollMedDollarVol  RollingSameVolCount
Ticker Date                                                                                                                              
A      1999-11-18     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN              NaN               NaN                  NaN
       1999-11-19  2.5074  2.5074  0.1037 -0.0824     NaN     NaN     NaN     NaN              NaN               NaN                  NaN
       1999-11-22  2.3577  2.4967  0.0948  0.0898     NaN     NaN     NaN     NaN              NaN               NaN                  NaN
       1999-11-23  2.3952  2.4895  0.1039 -0.0909 -0.0909     NaN     NaN     NaN              NaN               NaN                  NaN
       1999-11-24  1.1602  2.3945  0.0974  0.0266  0.0170     NaN     NaN     NaN              NaN               NaN                  NaN
...                  

In [ ]:
results_container, debug_container = plot_walk_forward_analyzer(
    df_ohlcv=df_ohlcv,
    precomputed_features=my_features, # Fast Features
    precomputed_close=my_close_matrix, # Fast Prices    
    default_start_date='2025-08-13',
    default_lookback=10,     # Was default_calc_period
    default_holding=5,       # Was default_fwd_period
    default_strategy='Sharpe (ATR)', # Was default_metric
    default_rank_start=15,
    default_rank_end=16,
    default_benchmark_ticker='SPY', 
    master_calendar_ticker='XOM',    
    quality_thresholds = { 
        'min_median_dollar_volume': 100_000, # A low "hard floor" to filter absolute errors/garbage
        # If min_liquidity_percentile is 0.8 (Top 20%), we want values > the 0.8 quantile.            
        'min_liquidity_percentile': 0.50,    # Dynamic: Only keep the top 50% of stocks by volume
        'max_stale_pct': 0.05, 
        'max_same_vol_count': 10
    },
    debug=True  # <-- Activate the new mode!
)

--- ⚙️ Initializing AlphaEngine ---
⚡ Using Pre-Computed Price Matrix (Instant Load)


FigureWidget({
    'data': [{'line': {'width': 2},
              'name': 'JPST',
              'type': 'scatter',
              'uid': '7379b3f3-2d7a-4e16-9282-c178c76a077f',
              'visible': True,
              'x': array([datetime.datetime(2025, 7, 30, 0, 0),
                          datetime.datetime(2025, 7, 31, 0, 0),
                          datetime.datetime(2025, 8, 1, 0, 0),
                          datetime.datetime(2025, 8, 4, 0, 0),
                          datetime.datetime(2025, 8, 5, 0, 0),
                          datetime.datetime(2025, 8, 6, 0, 0),
                          datetime.datetime(2025, 8, 7, 0, 0),
                          datetime.datetime(2025, 8, 8, 0, 0),
                          datetime.datetime(2025, 8, 11, 0, 0),
                          datetime.datetime(2025, 8, 12, 0, 0),
                          datetime.datetime(2025, 8, 13, 0, 0),
                          datetime.datetime(2025, 8, 14, 0, 0),
                          dateti

In [7]:
print_nested(results_container)
print('='*20)
print_nested(debug_container)

EngineOutput(portfolio_series=Date
2025-07-30    1.0000
2025-07-31    0.9914
2025-08-01    0.9986
2025-08-04    1.0150
2025-08-05    1.0230
2025-08-06    1.0248
2025-08-07    1.0255
2025-08-08    1.0314
2025-08-11    1.0285
2025-08-12    1.0398
2025-08-13    1.0651
2025-08-14    1.0578
2025-08-15    1.0558
2025-08-18    1.0532
2025-08-19    1.0656
2025-08-20    1.0674
dtype: float64, benchmark_series=Date
2025-07-30    1.0000
2025-07-31    0.9962
2025-08-01    0.9799
2025-08-04    0.9948
2025-08-05    0.9898
2025-08-06    0.9974
2025-08-07    0.9965
2025-08-08    1.0043
2025-08-11    1.0023
2025-08-12    1.0130
2025-08-13    1.0164
2025-08-14    1.0165
2025-08-15    1.0142
2025-08-18    1.0139
2025-08-19    1.0084
2025-08-20    1.0058
dtype: float64, normalized_plot_data=Ticker        JPST     LOW
Date                      
2025-07-30  1.0000  1.0000
2025-07-31  0.9998  0.9830
2025-08-01  1.0017  0.9955
2025-08-04  1.0019  1.0281
2025-08-05  1.0019  1.0441
2025-08-06  1.0019  1.0478
20

In [8]:
my_tickers = ['VOO', 'VLO', 'JPST']
date_start = '2025-08-13'
date_end = '2025-09-04'

# Create combined dictionary
combined_dict = create_combined_dict(
    df_ohlcv=df_ohlcv,
    features_df=my_features,
    tickers=my_tickers,
    date_start=date_start,
    date_end=date_end,
    verbose=False
)

print_nested(combined_dict)

VOO:
                Adj Open  Adj High  Adj Low  Adj Close    Volume      TR     ATR    ATRP       ROC_1       ROC_3   ROC_5  ROC_10  ROC_21  RollingStalePct  RollMedDollarVol  RollingSameVolCount
Date                                                                                                                                                                                        
2025-08-13   591.169   592.336  589.125    591.149   5408423   3.241  5.2072  0.0088  3.4867e-03  1.2120e-02  0.0193  0.0165  0.0367              0.0        2.9400e+09                  0.0
2025-08-14   589.235   591.817  588.816    591.159   6254536   3.001  5.0496  0.0085  1.6916e-05  1.4216e-02  0.0199  0.0204  0.0333              0.0        2.9409e+09                  0.0
2025-08-15   592.166   592.256  588.976    589.883   7216880   3.280  4.9232  0.0083 -2.1585e-03  1.3376e-03  0.0100  0.0352  0.0249              0.0        2.9461e+09                  0.0
2025-08-18   589.295   590.332  588.666    589

In [9]:
# 1. Access the data inside the container list
current_debug_data = debug_container[0]

# 2. Check if the audit data exists (it is created only in 'Ranking' mode)
if current_debug_data and 'audit_liquidity' in current_debug_data:
    audit = current_debug_data['audit_liquidity']
    snapshot_df = audit['universe_snapshot']
    
    print(f"📅 Date: {audit['date'].date()}")
    print(f"💰 Calculated Cutoff: ${audit['final_cutoff_usd']:,.0f}")
    print("-" * 30)

# 3. View the tickers right around the cutoff point
# Find the index where 'Passed_Vol_Check' switches from True to False
    try:
        # Get the integer location (iloc) of the last True value
        last_pass_iloc = np.where(snapshot_df['Passed_Vol_Check'])[0][-1]
        
        # Show 5 rows before and 5 rows after the cutoff
        start = max(0, last_pass_iloc - 5)
        end = min(len(snapshot_df), last_pass_iloc + 6)
        
        display(snapshot_df.iloc[start:end].style.format({
            'RollMedDollarVol': '${:,.0f}',
            'Calculated_Cutoff': '${:,.0f}',
            'RollingStalePct': '{:.1%}'
        }))
    except IndexError:
        print("Could not determine cutoff boundary (maybe all passed or all failed).")
        display(snapshot_df.head())
else:
    print("⚠️ No audit data found. Make sure you are in 'Ranking' mode and have clicked 'Update Chart'.")    

📅 Date: 2025-08-13
💰 Calculated Cutoff: $111,430,348
------------------------------


,TR,ATR,ATRP,ROC_1,ROC_3,ROC_5,ROC_10,ROC_21,RollingStalePct,RollMedDollarVol,RollingSameVolCount,Calculated_Cutoff,Passed_Vol_Check,Passed_Final
Ticker,,,,,,,,,,,,,,
TD,1.300000,0.840629,0.011287,0.017347,0.016792,0.014161,0.012507,0.014023,0.0%,"$111,982,661",0.000000,"$111,430,348",True,True
WCC,4.979000,6.014023,0.027975,0.019641,0.054533,0.044409,0.012452,0.087209,0.0%,"$111,830,156",0.000000,"$111,430,348",True,True
SPYV,0.507600,0.455391,0.008441,0.008934,0.016694,0.027877,0.018606,0.032774,0.0%,"$111,711,573",0.000000,"$111,430,348",True,True
GNRC,5.920000,5.932117,0.029244,0.020064,0.033631,0.046104,0.120718,0.381718,0.0%,"$111,583,772",0.000000,"$111,430,348",True,True
BR,3.825000,5.553364,0.021305,-0.003125,-0.016017,-0.019522,0.060810,0.119486,0.0%,"$111,542,641",0.000000,"$111,430,348",True,True
PATH,0.525000,0.361355,0.032880,0.046667,0.030957,-0.020499,-0.084929,-0.112278,0.0%,"$111,506,538",0.000000,"$111,430,348",True,True
BN,0.665800,0.981159,0.022251,0.010220,0.023489,-0.018669,-0.018379,0.045792,0.0%,"$111,354,159",0.000000,"$111,430,348",False,False
ESS,5.862000,6.316036,0.024650,0.018661,0.016261,0.016540,-0.036244,-0.077383,0.0%,"$111,199,576",0.000000,"$111,430,348",False,False
HEI,10.890000,8.083050,0.025830,-0.009935,-0.001468,-0.011405,-0.037849,-0.016407,0.0%,"$110,696,672",0.000000,"$111,430,348",False,False


In [10]:
snapshot_df.to_csv('./export_csv/snapshot_df.csv')
print(f"✅ Snapshot exported to: ./export_csv/snapshot_df.csv")
print(f"   Shape: {snapshot_df.shape}")
print(f"   Columns: {list(snapshot_df.columns)}")

✅ Snapshot exported to: ./export_csv/snapshot_df.csv
   Shape: (1619, 14)
   Columns: ['TR', 'ATR', 'ATRP', 'ROC_1', 'ROC_3', 'ROC_5', 'ROC_10', 'ROC_21', 'RollingStalePct', 'RollMedDollarVol', 'RollingSameVolCount', 'Calculated_Cutoff', 'Passed_Vol_Check', 'Passed_Final']


In [11]:
# Assuming you have run the variables setup from the previous step
snapshot_df = debug_container[0]['audit_liquidity']['universe_snapshot']

if 'AAPL' in snapshot_df.index:
    display(snapshot_df.loc[['AAPL']])
else:
    print("AAPL was not present in the data for this date.")

,TR,ATR,ATRP,ROC_1,ROC_3,ROC_5,ROC_10,ROC_21,RollingStalePct,RollMedDollarVol,RollingSameVolCount,Calculated_Cutoff,Passed_Vol_Check,Passed_Final
Ticker,,,,,,,,,,,,,,
AAPL,5.344,5.271,0.0226,0.016,0.0185,0.0954,0.1174,0.1171,0.0,1.0290e+10,0.0,1.1143e+08,True,True


In [ ]:
================================  
================================  
================================  

In [ ]:
df_ohlcv.loc['SPY']

In [ ]:
combined_dict['JPST']

In [ ]:
debug_container

In [ ]:
my_tickers = ['SPY', 'AAPL', 'IWM', 'QQQ', 'META', 'EEM', 'BABA']
my_tickers = ['NTES', 'LII',]


In [ ]:
display(snapshot_df)

In [ ]:
# Assuming your data is loaded in 'df_ohlcv'
print("--- 🚀 LAUNCHING APP ---")

# 1. We clear any old features so it calculates the new ROC columns
# 2. We pass 'debug=True' so we can see the audit logs in the console
results, debug_data = plot_walk_forward_analyzer(
    df_ohlcv, 
    precomputed_features=None, # Force recalculation of new features
    default_strategy='Pullback 5D', # Try one of your new strategies!
    debug=True
)